In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'


In [2]:
all_fights = pd.read_csv('Data Sets/Merged_Fights_Fighters.csv')
all_fights.drop(['Unnamed: 0'], axis=1, inplace=True)
#mask to remove contested fights
mask_no_contest_1 = ((all_fights.method != 'No Contest - Hoffman Failed Drug Test') & 
                   (all_fights.method != 'No Contest - Overturned by State Commission') &
                    (all_fights.method != 'No') & (all_fights.method != "NC") & 
                    (all_fights.method != 'No Contest - Caceres Failed Drug Test') & 
                     (all_fights.method != 'No Contest - Overturned by Commission' ) & 
                     (all_fights.method !='No Contest') & (all_fights.method != 'Draw'))
# df_contested fights
fights = all_fights[mask_no_contest_1]

In [3]:
method_code = []
for method in fights['method']:
    if method == 'Decision':
        x = 0
    elif method == 'TKO':
        x = 1
    elif method == 'Submission':
        x = 2
    elif method == 'KO':
        x = 1
    elif method == 'DQ':
        x = 3
    method_code.append(x)
fights['method_code'] = method_code

/Users/mikesanders/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
fights['f1nick_bin'] = fights['f1nick'].map(lambda x: 0 if x == 'None' else 1)
fights['f2nick_bin'] = fights['f2nick'].map(lambda x: 0 if x == 'None' else 1)

In [7]:
fights['f1nick_bin'] = temp
fights['f1nick_bin'].value_counts()

/Users/mikesanders/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


1    2630
0     880
Name: f1nick_bin, dtype: int64

In [8]:
fights['f2nick_bin'] = temp2
fights['f2nick_bin'].value_counts()

/Users/mikesanders/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


1    2550
0     960
Name: f2nick_bin, dtype: int64

In [10]:
#Starting my models
import statsmodels.formula.api as smf
import patsy
formula = 'method_code ~ C(ref) + C(f1name) + f1height + f1weight + C(f1association) + C(f1class) + C(f1locality) + C(f1country) + C(f2name) + f2height + f2weight + C(f2association) + C(f2class) + C(f2locality) + C(f2country) + f1age + f2age + f1nick_bin + f2nick_bin - 1'
Y,X = patsy.dmatrices(formula, data=fights, return_type = "dataframe")
y = np.ravel(Y)
print X.shape
print y.shape

(3510, 5128)
(3510,)


In [11]:
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

Cs = 1.0/np.logspace(-1,4,20)

search_parameters_lr = {
    'penalty': ['l1','l2'],
    'C': Cs,
    'class_weight': [None, 'balanced'],
    'solver': ['liblinear'],
    'n_jobs': [4]
    }

lr_est = GridSearchCV(lr, search_parameters_lr, cv=5)

In [12]:
lr_grid = lr_est.fit(X,y)
print lr_grid.best_params_
print lr_grid.best_score_

{'penalty': 'l1', 'C': 0.012742749857031348, 'n_jobs': 4, 'solver': 'liblinear', 'class_weight': None}
0.438461538462


In [ ]:
lr_best = LogisticRegression(penalty= 'l1', C= 0.012742749857031348, n_jobs = 4, solver= 'liblinear', class_weight= None)
lr_best.fit(X,y)
lr_best.co

In [ ]:
y_pred_lr = lr_best.predict(X)
print classification_report(y, y_pred_lr)

In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
class multi_roc:
    
    # model / estimator
    estimator = False
    
    
    def __init__(self, estimator):
        self.estimator = estimator
        
        if "classes_" not in vars(estimator):
            raise Exception("EstimatorError", "Estimator has no fit data.  Run .fit()")
                            
    def plot(self, X_test, y_test, filter_class=False, figsize=(10,10)):
        _ = ""
        
        y_score = self.estimator.decision_function(X_test)
        y = label_binarize(y_test, classes=self.estimator.classes_)
        
        n_classes = y.shape[1]
        
        fpr = dict()
        tpr = dict()
        
        roc_auc = dict()
        
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y[:, i], y_score[:, i])
            roc_auc[self.estimator.classes_[i]] = auc(fpr[i], tpr[i])
        
        # Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(y.ravel(), y_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
        
        if filter_class:
            # Plot of a ROC curve for a specific class
            figure = plt.figure()
            figure.set_size_inches(figsize[0], figsize[1])
            plt.plot(fpr[2], tpr[2], label='ROC curve (area = %0.2f)' % roc_auc[2], figsize=(10,10))
            plt.plot([0, 1], [0, 1], 'k--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title('Receiver operating characteristic example')
            plt.legend(loc="lower right")
            plt.show()
            
            return
        
        # Plot ROC curve
        plt.figure(figsize=figsize)
        plt.plot(fpr["micro"], tpr["micro"],
                 label='micro-average ROC curve (area = {0:0.2f})'
                       ''.format(roc_auc["micro"]))
        
        for i in range(n_classes):
            plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                           ''.format(i, roc_auc[i]))
        
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Some extension of Receiver operating characteristic to multi-class')
        plt.legend(loc="lower right")
        plt.show()

        


roc = multi_roc(estimator = lr_best)
roc.plot(X, y, figsize=(10,10))

In [14]:
from sklearn.grid_search import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

search_parameters_knn = {
    
    'weights': ['uniform', 'distance'],
    'algorithm' : ['auto'],
    'leaf_size': [5, 10, 50,50],
    'n_neighbors': [1,3,5,7],
    'n_jobs': [4],
    'p': [2],
    'metric': ['minkowski']
    }

knn_est = GridSearchCV(knn, search_parameters_knn, cv=5)

In [15]:
knn_grid = knn_est.fit(X,y)
print knn_grid.best_params_
print knn_grid.best_score_

{'n_neighbors': 7, 'n_jobs': 4, 'algorithm': 'auto', 'metric': 'minkowski', 'p': 2, 'weights': 'uniform', 'leaf_size': 5}
0.376068376068


In [16]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

search_parameters_rfc = {
    'n_estimators': [10,20,30],
    'criterion': ['gini', 'entropy'],
    'max_depth': [1,3,5, None],
    'max_features': ['auto', 'sqrt', 'log2', None]
    }
rfc_est = GridSearchCV(rfc, search_parameters_rfc, cv=5)

In [17]:
rfc_grid = rfc_est.fit(X,y)
print rfc_grid.best_params_
print rfc_grid.best_score_

{'max_features': None, 'n_estimators': 30, 'criterion': 'gini', 'max_depth': 1}
0.435612535613


In [18]:
rfc_best = RandomForestClassifier(max_features= None, n_estimators = 30, criterion = 'gini', max_depth = 1)

In [ ]:
rfc